# Questão de Negócio

A empresa Star Jeans! Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. 

Depois de vários negócio bem sucedidos, eles estão planejando entrar no mercado de moda
dos USA como um modelo de negócio do tipo E-commerce.

A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso
o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação
baixo e escalar a medida que forem conseguindo clientes.

Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência
nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e
o material para a fabricação de cada peça.

Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes
perguntas: 
1. Qual o melhor preço de venda para as calças? 
2. Quantos tipos de calças e suas cores para o produto inicial? 
3. Quais as matérias-prima necessárias para confeccionar as calças?

As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

# 0.0 Imports


In [1]:
import requests
import inflection
import regex as re
import numpy as np
import pandas as pd
import seaborn as sns

from datetime import datetime
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from IPython.core.display import HTML

## 0.1 Helper Functions 

In [2]:
def jupyter_settings():
    %matplotlib inline
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25,12]
    plt.rcParams['font.size'] = 24
    plt.rcParams['figure.dpi'] = 100
    
    display( HTML( '<style>.container{width:100% !important; }</style>'))
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    
    # ignora future warnings
    #warnings.filterwarnings('ignore')
    
    sns.set()

In [3]:
jupyter_settings()

# 1.0 Extração de dados em HTML

## 1.1 Make collect from one page

In [4]:
# make request
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 {Macintosh; Intel Mac Os X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36}'}
page = requests.get(url, headers=headers)

In [5]:
# Instance Beautiful Soup
soup = BeautifulSoup(page.text, "html.parser")

In [6]:
# extract all products
products = soup.find('ul', class_='products-listing small')

In [7]:
product_list = products.find_all('article', class_='hm-product-item')

# product id
product_id = [p.get('data-articlecode') for p in product_list]

# product_category
product_category = [p.get('data-category') for p in product_list]

In [8]:
# product name
product_list = products.find_all('a', class_='link')
product_name = [p.get_text() for p in product_list]

In [9]:
# price
product_list = products.find_all('span', class_='price regular')
product_price = [p.get_text() for p in product_list]

In [10]:
# product color

In [11]:
# product composition

In [12]:
# merge scrapy into data frame
data = pd.DataFrame([product_id, product_category, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [13]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20
1,1024256002,men_jeans_slim,Slim Jeans,$ 24.99,2022-07-30 14:00:20
2,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-07-30 14:00:20
3,1013317006,men_jeans_joggers,Hybrid Regular Tapered Joggers,$ 44.99,2022-07-30 14:00:20
4,0875105023,men_jeans_relaxed,Relaxed Jeans,$ 29.99,2022-07-30 14:00:20


## 1.2 Request products from all pages

In [14]:
# collect all products available

total_item = soup.find_all('h2', class_='load-more-heading')[0].get('data-total')
total_item

'77'

In [15]:
# detect total page numbers
page_number = np.ceil(int(total_item)/36)
page_number

3.0

In [16]:
# create link from total of page numbers
url02 = url + '?page-size' + str(int(page_number*36))
url02

'https://www2.hm.com/en_us/men/products/jeans.html?page-size108'

## 1.3 Collect product collor

### Testing One Product


In [17]:
#Api Request
url03 = 'https://www2.hm.com/en_us/productpage.0690449022.html'

page03 = requests.get(url03, headers=headers)

# Beautiful Soup
soup03 = BeautifulSoup(page03.text, 'html.parser')

In [18]:
# color name
product_list = soup03.find_all('a', {'class':['filter-option miniature', 'filter-option miniature active']} )
color_name = [p.get('data-color') for p in product_list]

# product id
product_id = [p.get('data-articlecode') for p in product_list]

df_color = pd.DataFrame( [product_id, color_name]).T
df_color.columns = ['product_id', 'color_name']

# generate style id + color id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

In [19]:
# composition

product_composition_list = soup03.find_all('div', class_='details-attributes-list-item')
product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

# rename dataframe
df_composition = pd.DataFrame(product_composition).T
df_composition.columns = df_composition.iloc[0]

# delete first row
df_composition = df_composition.iloc[1:].fillna(method='ffill')

# generate style id + color id
df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

# merge data color + decomposition
data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')

In [20]:
data_sku

,product_id,color_name,style_id,color_id,Fit,Composition
0,0690449001,Light denim blue/trashed,0690449,001,Skinny fit,"Cotton 98%, Spandex 2%"
1,0690449001,Light denim blue/trashed,0690449,001,Skinny fit,Lining: Polyester 100%
2,0690449001,Light denim blue/trashed,0690449,001,Skinny fit,Lining: Polyester 100%
3,0690449001,Light denim blue/trashed,0690449,001,Skinny fit,Lining: Polyester 100%
4,0690449001,Light denim blue/trashed,0690449,001,Skinny fit,Lining: Polyester 100%
...,...,...,...,...,...,...
109,0690449059,Denim blue,0690449,059,Skinny fit,Lining: Polyester 100%
110,0690449059,Denim blue,0690449,059,Skinny fit,Lining: Polyester 100%
111,0690449059,Denim blue,0690449,059,Skinny fit,Lining: Polyester 100%
112,0690449059,Denim blue,0690449,059,Skinny fit,Lining: Polyester 100%


### All Product


In [21]:
# Empty dataframe
df_details = pd.DataFrame()

# unique columns for all products
aux = []

cols = {'Art. No.', 'Care instructions', 'Composition', 'Concept', 'Description', 'Fit', 'Imported',
 'Material', 'More sustainable materials', 'Nice to know', 'Size', 'color_id', 
 'messages.clothingStyle', 'messages.garmentLength', 'messages.waistRise', 'style_id'}
df_pattern = pd.DataFrame(columns=cols)

for i in range(len(data)):
    #Api Request
    url03 = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'

    page03 = requests.get(url03, headers=headers)

    # Beautiful Soup
    soup03 = BeautifulSoup(page03.text, 'html.parser')

    # color name
    product_list = soup03.find_all('a', {'class':['filter-option miniature', 'filter-option miniature active']} )
    color_name = [p.get('data-color') for p in product_list]

    # product id
    product_id = [p.get('data-articlecode') for p in product_list]

    df_color = pd.DataFrame( [product_id, color_name]).T
    df_color.columns = ['product_id', 'color_name']

    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

    # composition

    product_composition_list = soup03.find_all('div', class_='details-attributes-list-item')
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

    # rename dataframe
    df_composition = pd.DataFrame(product_composition).T
    df_composition.columns = df_composition.iloc[0]

    # delete first row
    df_composition = df_composition.iloc[1:].fillna(method='ffill')
    
    # garantee the same number of columns
    df_composition = pd.concat([df_pattern, df_composition], axis=0)
    
    # generate style id + color id
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

    aux = aux + df_composition.columns.tolist()
    
    # merge data color + decomposition
    data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'Size', 'Description'
                                                 , 'Material']], how='left', on='style_id')
    
    # all details products
    df_details = pd.concat([df_details, data_sku], axis=0)

# Join showroom data + details
data['style_id'] = data['product_id'].apply(lambda x: x[:-3])
data['color_id'] = data['product_id'].apply(lambda x: x[-3:])

data_raw = pd.merge(data, df_details[['style_id', 'Fit', 'Composition', 'Size', 'Description',
                                    'Material']],how='left', on='style_id')

In [22]:
data_raw.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,Fit,Composition,Size,Description,Material
0,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Light denim blue,Denim
1,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Solid-color,Denim
2,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Solid-color,Denim
3,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Solid-color,Denim
4,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Solid-color,Denim


# 2.0 Descrição dos dados

In [35]:
data1 = data_raw.copy()

In [24]:
data1.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,Fit,Composition,Size,Description,Material
0,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Light denim blue,Denim
1,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Solid-color,Denim
2,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Solid-color,Denim
3,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Solid-color,Denim
4,0971061004,men_jeans_slim,Slim Tapered Cropped Jeans,$ 29.99,2022-07-30 14:00:20,0971061,004,Slim fit,"Cotton 99%, Spandex 1%",NaN,Solid-color,Denim


## 2.1 Rename Columns

In [36]:
old_cols = ['product_id', 'product_category', 'product_name', 'product_price',
       'scrapy_datetime', 'style_id', 'color_id', 'Fit', 'Composition', 'Size',
       'Description', 'Material']

snakecase = lambda x : inflection.underscore(x)

new_cols = list(map(snakecase, old_cols))

data1.columns = new_cols

## 2.2 Data dimension

In [26]:
print(f'Number of rows: {data1.shape[0]}')
print(f'Number of columns: {data1.shape[1]}')

Number of rows: 8751
Number of columns: 12


## 2.3 Check NA

In [27]:
data1.isna().sum()

product_id             0
product_category       0
product_name           0
product_price          0
scrapy_datetime        0
style_id               0
color_id               0
fit                    0
composition            0
size                7932
description            0
material               0
dtype: int64

## 2.4 Fillout NA

In [28]:
# check type of data
data1['size'].unique()

array([nan, 'Inner leg: Length: 89.5 cm (Size 33)',
       'Waist: Circumference: 90.5 cm (Size 33)'], dtype=object)

In [37]:
# Create Collumn Inner Leg
data1['inner_leg_size'] = data1['size'].apply(lambda x: 89.5 if x=='Inner leg: Length: 89.5 cm (Size 33)'
                                              else 0)

# Create collumn Waist
data1['waist_size'] = data1['size'].apply(lambda x: 90.5 if x=='Waist: Circumference: 90.5 cm (Size 33)'
                                              else 0)

# fillout size value
data1['size'] = data1['size'].apply(lambda x: 0 if pd.isnull(x) else 33)

In [30]:
# check result
data1.isna().sum()

product_id          0
product_category    0
product_name        0
product_price       0
scrapy_datetime     0
style_id            0
color_id            0
fit                 0
composition         0
size                0
description         0
material            0
inner_leg_size      0
waist_size          0
dtype: int64

## 2.5 Cleaning Data

In [38]:
# product_name - undercase and removing special characters
data1['product_name'] = data1['product_name'].apply(lambda x: x.replace(' ', '_').replace('®', '').lower())

# price - removing "$"
data1['product_price'] = data1['product_price'].apply(lambda x: x.replace('$ ', ''))

# fit - undercase and removing space
data1['fit'] = data1['fit'].apply(lambda x: x.replace(' ', '_').lower())

In [39]:
# Cleaning composition

# Remove composition from details in products
data1 = data1[~data1['composition'].str.contains('Shell', na=False)]
data1 = data1[~data1['composition'].str.contains('Pocket lining', na=False)]
data1 = data1[~data1['composition'].str.contains('Pocket', na=False)]
data1 = data1[~data1['composition'].str.contains('Lining', na=False)]

# break composition by comma
dfaux = data1['composition'].str.split(',', expand=True)

# choose order from columns
df_ref = pd.DataFrame(index=np.arange(len(data1)), columns=['cotton', 'spandex', 'polyester'])

# cotton - collect all data
df_cotton = dfaux[0]
df_cotton.name = 'cotton'

df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# spandex - collect all data
df_spandex = dfaux.loc[dfaux[1].str.contains('Spandex', na=True), 1]
df_spandex.name = 'spandex'

df_ref = pd.concat([df_ref, df_spandex], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

df_spandex2 = dfaux.loc[dfaux[2].str.contains('Spandex', na=True), 2]
df_spandex2.name = 'spandex2'

df_ref = pd.concat([df_ref, df_spandex2], axis=1)
df_ref['spandex'] = df_ref.apply(lambda x: x['spandex'] if pd.isna(x['spandex2']) else x['spandex2'], axis=1)
df_ref.drop(columns='spandex2', inplace=True)

# polyester - collect all data
df_polyester = dfaux.loc[dfaux[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# drop all rows with NA
df_ref.dropna(axis=0, how='all', inplace=True)

# final join
data1 = pd.concat([data1, df_ref], axis=1)

# format composition data
data1['cotton'] = data1['cotton'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)
data1['spandex'] = data1['spandex'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)
data1['polyester'] = data1['polyester'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

# cleaning NA in polyester
data1['polyester'].fillna(0, inplace=True)

In [40]:
# description - undercase and removing space
data1['description'] = data1['description'].apply(lambda x: x.replace(' ', '_').lower())

# material - undercase and removing space
data1['material'] = data1['material'].apply(lambda x: x.replace(' ', '_').lower())

## 2.6 Data types

In [41]:
# checking types
data1.dtypes

product_id           object
product_category     object
product_name         object
product_price        object
scrapy_datetime      object
style_id             object
color_id             object
fit                  object
composition          object
size                  int64
description          object
material             object
inner_leg_size      float64
waist_size          float64
cotton               object
spandex              object
polyester            object
dtype: object

## 2.6 Change Types

In [44]:
# product_id
data1['product_id'] = data1['product_id'].astype('int64')

# product_price
data1['product_price'] = data1['product_price'].astype('float64')

# scrapy_datetime
data1['scrapy_datetime'] = pd.to_datetime(data1['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# style_id
data1['style_id'] = data1['style_id'].astype('int64')

# color_id
data1['color_id'] = data1['color_id'].astype('int64')

# cotton
data1['cotton'] = data1['cotton'].astype('int64')

# spandex
data1['spandex'] = data1['spandex'].astype('int64')

# polyester
data1['polyester'] = data1['polyester'].astype('int64')

In [45]:
# check result
data1.dtypes

product_id                   int64
product_category            object
product_name                object
product_price              float64
scrapy_datetime     datetime64[ns]
style_id                     int64
color_id                     int64
fit                         object
composition                 object
size                         int64
description                 object
material                    object
inner_leg_size             float64
waist_size                 float64
cotton                       int64
spandex                      int64
polyester                    int64
dtype: object